In [ ]:
import numpy as np
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
import pandas as pd
import scanpy as sc
import anndata
import os
import sys

In [ ]:
sys.path.insert(1, '/secure/projects/tacco/')
import tacco as tc

In [ ]:
sys.path.insert(1, '/secure/projects/HTAPP_MBC/src/spatial')
from spatial_utils import *

In [ ]:
combined_dir=out_dir_base+"/combined_all/"
os.makedirs(combined_dir, exist_ok=True)

In [ ]:
sample_sheet=pd.read_csv(codeDir+"/meta/sampletracking_HTAPP_MBC_spatial.csv")

In [ ]:
#sc sample annotation
sc_samples=pd.read_excel(codeDir+"/meta/clinical features2.xlsx")
sc_samples['sample']=sc_samples['htapp_id']+'-'+sc_samples['htapp_caseid']

In [ ]:
sc_samples.head()

In [ ]:
#These identifiers are set as defaults in many functions. If names are changed, this has to be adapted when calling the functions.
ct_column='cell_type'
sm_methods=["slide_seq","merfish","merfish_bin","codex","exseq","exseq_bin"] #spatial mathod names are hard-coded for now because some methods need special treatments. Can be fixed by giving eatch method it's own variable.
sc_method="scRNAseq"
all_methods=[sc_method,*sm_methods]
all_methods

In [ ]:
run_samples=list(filter(lambda a: not a in [''], list(sample_sheet['sample'].unique()))) 
run_samples

In [ ]:
sc_samples=sc_samples.loc[sc_samples['sample'].isin(run_samples)][['sample','method','receptors_biopsy','site_biopsy']].set_index('sample')

In [ ]:
sc_samples

In [ ]:
adatas_combined=pd.Series([[] for i in range(len(all_methods))],index=all_methods,dtype=object) 
#add combined scRNAseq that was produced in the MERFISH analysis 
adatas_combined['scRNAseq']=sc.read_h5ad(out_dir_base+"/combined_merfish/scRNAseq.h5ad")
for method in sm_methods:
    adatas_combined[method]=sc.read(combined_dir+"/"+method+".h5ad",compression="gzip")

In [ ]:
adatas_combined

In [ ]:
for method in sm_methods:
    adatas_combined[method].obsm['OT'].fillna(0,inplace=True)
    adatas_combined[method].obsm['RCTD'].fillna(0,inplace=True)
    adatas_combined[method].obsm['OT_max'].fillna(0,inplace=True)

for method in all_methods:    
    for col in sc_samples.columns:
        adatas_combined[method].obs[col]=adatas_combined[method].obs['sample'].map(sc_samples[col])

In [ ]:
sel_colors=colors[colors.index.isin(adatas_combined[sc_method].obs[ct_column])]
keys=pd.Series([['OT']]*len(adatas_combined.index), index=adatas_combined.index)
keys['scRNAseq'] = ['cell_type']
fig = tc.pl.scatter(adatas_combined, keys=keys,colors=sel_colors,joint=True,position_key="X_umap",axsize=(6,6), share_scaling=False, padding=0.75);
figheight = fig.get_size_inches()[1]
fig.savefig(combined_dir+'/all_UMAP_OT.pdf',bbox_inches='tight')

In [ ]:
fig = tc.pl.scatter(adatas_combined, keys=['sample'],joint=True,position_key="X_umap",axsize=(6,6), share_scaling=False, padding=0.75);
fig.savefig(combined_dir+'/all_UMAP_sample.pdf',bbox_inches='tight')

In [ ]:
keys=pd.Series([['replicate']]*len(adatas_combined.index), index=adatas_combined.index)
keys['scRNAseq'] = ['method']
fig = tc.pl.scatter(adatas_combined, keys=keys,joint=True,position_key="X_umap",axsize=(6,6), share_scaling=False, padding=0.75);
fig.savefig(combined_dir+'/all_UMAP_repl.pdf',bbox_inches='tight')

In [ ]:
fig = tc.pl.scatter(adatas_combined, keys=['leiden'],joint=True,position_key="X_umap",axsize=(6,6), share_scaling=False, padding=0.75);
fig.savefig(combined_dir+'/all_UMAP_leiden.pdf',bbox_inches='tight')

In [ ]:
fig = tc.pl.scatter(adatas_combined, keys=['site_biopsy'],joint=True,position_key="X_umap",axsize=(6,6), share_scaling=False, padding=0.75);
fig.savefig(combined_dir+'/all_UMAP_site.pdf',bbox_inches='tight')

In [ ]:
fig = tc.pl.scatter(adatas_combined, keys=['receptors_biopsy'],joint=True,position_key="X_umap",axsize=(6,6), share_scaling=False, padding=0.75);
fig.savefig(combined_dir+'/all_UMAP_receptor.pdf',bbox_inches='tight')

In [ ]:
df=pd.DataFrame(adatas_combined['merfish'].obsm['X_pca']).set_index(adatas_combined['merfish'].obs.index)

In [ ]:
df['OT']=adatas_combined['merfish'].obs['OT']

In [ ]:
df_sub=df.iloc[:].sort_values("OT")
sel_colors=colors[colors.index.isin(df_sub['OT'])].sort_index()
matplotlib.rcParams['figure.dpi'] = low_dpi
fig,ax=plt.subplots(figsize=[10,5])
pd.plotting.parallel_coordinates(df_sub, class_column='OT',cols=np.arange(10),color=sel_colors,ax=ax,linewidth=0.1)
ax.legend(bbox_to_anchor=(1.0, 1.0))
ax.hlines(0,xmin=0,xmax=9)
matplotlib.rcParams['figure.dpi'] = high_dpi